# Neural Network Prediction

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# read prepared data from previous notebooks
X = pd.read_csv('X.csv', index_col=0)
Y = pd.read_csv('Y.csv', index_col=0)
print(X.shape)
print(Y.shape)

(2985, 885)
(2985, 1)


Visualize X and Y dataset

In [3]:
X.head()

,mol_weight,log_p,num_h_donors,num_h_acceptors,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,311.422,3.3188,1.0,2.0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,299.461,3.2412,1.0,3.0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,311.422,3.3188,1.0,2.0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,327.877,3.8331,1.0,2.0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,372.328,3.9422,1.0,2.0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
Y.head()

,pIC50
0,5.301030
1,5.568636
2,5.744727
3,4.958607
4,5.000000


## Building Neural Network

Train Test Split
Splitting the dataset into training and test sets.

In [5]:
# Get 80% of the dataset as the training set. Put the remaining 20% as the test set: x_test and y_test.
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)

# Transpose the dataset
x_train = np.array(x_train.T)
x_test = np.array(x_test.T)
y_train =np.array(y_train.T)
y_test = np.array(y_test.T)

# Every column is one sample of chemical compound with a dimension of 885
# There are in total 2388 training samples
print(x_train.shape)
print(y_train.shape)

(885, 2388)
(1, 2388)


## Defines the input size, hidden size and output size

In [6]:
input_size = 885
hidden_size = 128
output_size = 1

Initialize parameters

In [7]:
def init_param(input_size, hidden_size, output_size):
    W1 = np.random.randn(hidden_size, input_size) * 0.01
    b1 = np.random.randn(hidden_size, 1) * 0.01
    W2 = np.random.randn(output_size, hidden_size) * 0.01
    b2 = np.random.randn(output_size, 1) * 0.01
    return W1, b1, W2, b2

Define Relu

In [8]:
def ReLU(Z):
    return np.maximum(Z, 0)

## Forward Propagation

In [9]:
def forward_prop(X, W1, B1, W2, B2):
    Z1 = np.dot(W1, X) + B1
    A1 = ReLU(Z1)
    Z2 = np.dot(W2, A1) + B2
    A2 = Z2 # linear activation
    return Z1, A1, Z2, A2

## Model Dimensions
$$
W1:128*885 \\
X:885*2388 \\
B1:128*1 \\
Z1:128*2388 \\
A1:128*2388 \\
W2:1*128 \\ 
B2: 1*1 \\
Z2: 1*2388 \\ 
A2: 1*2388 \\ 
Y:1*2388
$$

## Forward Propagation:
$$
Z1 = W1*X + B1
\\
A1 = ReLU(Z1)
\\
\\
Z2 = W2*A1 + B2
\\
A2 = Z2
$$

## Loss Function (MSE) and Back Propagation
$$
Loss(A2, Y) = \frac{1}{2*2388} \sum_{i=1}^2388 (A_2 - Y) ^ 2
$$

Gradients:

$$
dZ_2 = A_2 - Y \\

$$

## Define derivative of ReLU and back prop

In [10]:
def deriv_ReLU(Z):
    return Z > 0

def back_prop(X, Y, Z1, A1, Z2, A2, W2):
    m = X.shape[1]
    
    # Output layer gradients
    dZ2 = A2 - Y
    dW2 = (1 / m) * np.dot(dZ2, A1.T)
    dB2 = (1 / m) * np.sum(dZ2)
    
    # Hidden layer gradients
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = dA1 * deriv_ReLU(Z1)
    dW1 = (1 / m) * np.dot(dZ1, X.T)
    dB1 = (1 / m) * np.sum(dZ1)
    
    return dW1, dB1, dW2, dB2

## Define loss function

In [11]:
def compute_loss(Y, A2):
    m = Y.shape[1]
    loss = (1 / (2 * m)) * np.sum((A2 - Y) ** 2)
    return loss

## Define Gradient descent to update parameters

In [12]:
def update_parameters(W1, B1, W2, B2, dW1, dB1, dW2, dB2, learning_rate):
    W1 -= learning_rate * dW1
    B1 -= learning_rate * dB1
    W2 -= learning_rate * dW2
    B2 -= learning_rate * dB2
    return W1, B1, W2, B2

## Define Training loop

In [13]:
def train(X, Y, learning_rate=0.0001, epochs=1000):
    W1, B1, W2, B2 = init_param(input_size = 885, hidden_size = 128, output_size = 1)
    
    for epoch in range(epochs):
        Z1, A1, Z2, A2 = forward_prop(X, W1, B1, W2, B2)
        loss = compute_loss(Y, A2)
        
        
        dW1, dB1, dW2, dB2 = back_prop(X, Y, Z1, A1, Z2, A2, W2)
        W1, B1, W2, B2 = update_parameters(W1, B1, W2, B2, dW1, dB1, dW2, dB2, learning_rate)
        
        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss}')
    
    return W1, B1, W2, B2

## Starts training

In [14]:
trained_W1, trained_B1, trained_W2, trained_B2 = train(x_train, y_train)

Epoch 0, Loss: 19.53322752570043


Epoch 100, Loss: 1.3053604379629051


Epoch 200, Loss: 1.3017362147972464


Epoch 300, Loss: 1.2981095773489093


Epoch 400, Loss: 1.2944692660832624


Epoch 500, Loss: 1.2908065775812898


Epoch 600, Loss: 1.2871118041923082


Epoch 700, Loss: 1.2833796033136433


Epoch 800, Loss: 1.279599821577683


Epoch 900, Loss: 1.2757640150555045


## With 1000 epoch of training, the loss on the training set is 0.62

Test the model on the test set

In [15]:
_, _, _, predicted_A2 = forward_prop(x_test, trained_W1, trained_B1, trained_W2, trained_B2)
print(f"Prediction : {predicted_A2[0][:50]}")
print(f"Actual: {y_test[0][:50]}")
loss = compute_loss(y_test, predicted_A2)
print(f"loss : {loss}")

Prediction : [4.87482357 4.92331901 3.09557398 4.92331901 4.92331901 4.92331901
 4.55514306 4.92331901 4.92331901 5.42733867 9.12946456 6.78523428
 6.49672711 3.09557398 7.23893633 7.86227152 4.2960192  4.71589222
 5.8079007  4.92331901 4.92331901 4.92331901 6.39641101 4.76629884
 5.0517421  4.60670681 4.92331901 5.601574   5.74495639 4.92331901
 4.92331901 4.89313382 4.92331901 4.92331901 3.87192008 7.20371088
 3.55477797 7.82910736 4.89233873 4.9757123  5.46480299 3.1351524
 4.88109024 5.68783912 3.89935126 4.92331901 4.92331901 7.37832879
 5.09516897 6.84564387]
Actual: [6.32239305 5.         4.82073554 6.20065945 5.48148606 5.04575749
 4.70114692 4.72746222 5.08249449 8.52287875 5.52287875 4.61618463
 5.07572071 4.92481815 8.55284197 4.39794001 4.82102305 4.00674017
 5.95860731 5.46852108 5.1948391  4.78914663 5.         4.09151498
 4.85387196 4.2756422  4.69897    7.25289907 6.09691001 4.66134433
 4.69897    5.         4.73873713 4.73873713 7.49485002 5.
 4.60205999 7.26760624 6.8

## Results

With 597 samples in the test set, the total loss is 1.18